### 1. Data processing and Training Model

* Configurations

In [ ]:
import sys
from os.path import dirname, abspath

import numpy as np
import pandas as pd
import pickle as pkl

from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

* Data Processing

In [ ]:
np.random.seed(0)
parent = dirname(dirname(abspath('')))
sys.path.append(parent)
german_data = get_and_preprocess_german()

X_values = german_data["x_values"]
y_values = german_data["y_values"]

scalar = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(
    X_values, y_values, test_size=0.20)
cols = X_train.columns

X_train['y'] = y_train
X_test['y'] = y_test
X_train.to_csv('c:\\Users\\Dell V3400\\OneDrive\\Tài liệu\\machine learning\\ML1 Project\\data\\german_train.csv')
X_test.to_csv('c:\\Users\\Dell V3400\\OneDrive\\Tài liệu\\machine learning\\ML1 Project\\data\\german_test.csv')
X_train.pop("y")
X_test.pop("y")

X_train = X_train.values
X_test = X_test.values

* Model Training

In [ ]:
cat_pipeline = Pipeline([('scaler', StandardScaler()),
                        ('cat', CatBoostClassifier())])
cat_pipeline.fit(X_train, y_train)

print("Train Score:", cat_pipeline.score(X_train, y_train))
print("Score:", cat_pipeline.score(X_test, y_test))
print("Portion y==1:", np.sum(y_test == 1)
      * 1. / y_test.shape[0])

print("Column names: ", cols)
# print("Coefficients: ", lr_pipeline.named_steps["lr"].coef_)

with open("./data/german_model_grad_tree.pkl", "wb") as f:
    pkl.dump(cat_pipeline, f)

print("Saved model!")

38
Train Score: 0.8825
Score: 0.745
Portion y==1: 0.71
Column names:  Index(['Gender', 'ForeignWorker', 'Single', 'Age', 'LoanDuration',
       'LoanAmount', 'LoanRateAsPercentOfIncome', 'YearsAtCurrentHome',
       'NumberOfOtherLoansAtBank', 'NumberOfLiableIndividuals', 'HasTelephone',
       'CheckingAccountBalanceGreaterOrEqualTo0',
       'CheckingAccountBalanceGreaterOrEqualThan200',
       'SavingsAccountBalanceGreaterOrEqualThan200',
       'SavingsAccountBalanceGreaterOrEqualThan500', 'MissedPayments',
       'NoCurrentLoan', 'CriticalAccountOrLoansElsewhere', 'OtherLoansAtBank',
       'OtherLoansAtStore', 'HasCoapplicant', 'HasGuarantor', 'OwnsHouse',
       'RentsHouse', 'Unemployed', 'YearsAtCurrentJobLessThan1',
       'YearsAtCurrentJobGreaterOrEqualThan4', 'JobClassIsSkilled',
       'loanpurposeBusiness', 'loanpurposeEducation', 'loanpurposeElectronics',
       'loanpurposeFurniture', 'loanpurposeHomeAppliances',
       'loanpurposeNewCar', 'loanpurposeOther', 'loanpur

### 2. Explaining

* Configurations

In [ ]:
from explainer import Explainer
from explanation_methods.anchor_explainer import AnchorExplainer
from explanation_methods.lime_explainer import Lime
from explanation_methods.shap_explainer import SHAPExplainer
from explanation_methods.perturbation_methods import NormalPerturbation
from faithfulness_sorter import FaithfulnessSorter, Explanation

* Load model

In [8]:
X_train = pd.read_csv("c:\\Users\\Dell V3400\\OneDrive\\Tài liệu\\machine learning\\ML1 Project\\data\\german_train.csv")

In [ ]:
X_train_np = X_train
feature_names = X_train.columns.tolist()
discrete_features = [i for i, col in enumerate(X_train.columns) if 'categorical' in col]  # Assuming categorical features

In [12]:
# Drop unnecessary columns
if 'y' in feature_names:
    feature_names.remove('y')  # Remove target variable
if 'Unnamed: 0' in feature_names:
    feature_names.remove('Unnamed: 0')  # Remove index column

# Ensure the dataset matches the feature names
X_train_np = X_train_np[:, :len(feature_names)]  # Align shape with valid features

In [6]:
with open("c:\\Users\\Dell V3400\\OneDrive\\Tài liệu\\machine learning\\ML1 Project\\data\\german_model_grad_tree.pkl", "rb") as f:
   model=pkl.load(f)

In [ ]:
def model_callable(data):
    return model.predict_proba(data)

* Getting explainations

In [13]:
explainer = Explainer(explanation_dataset=X_train_np,
                      explanation_model=model_callable,
                      feature_names=feature_names,
                      discrete_features=discrete_features)

In [ ]:
# Access the first row from X_test as a NumPy array and reshape it
sample_data = X_test[0].reshape(1, -1)

In [ ]:
mega_explanation = explainer.explain_instance(data=sample_data)

In [17]:
print(f"Best Explanation Method: {mega_explanation[0].best_explanation_type}")
print(f"Explanation Score: {mega_explanation[0].score}")
print(f"Feature Importance: {mega_explanation[0].list_exp}")

Best Explanation Method: lime_0.25
Explanation Score: 0.4667754962888392
Feature Importance: [ 4.3687555e-03 -1.9995717e-03  3.2969711e-03 -1.3142395e-03
  1.3283708e-02  5.1954640e-03  5.7685506e-03  3.6398688e-04
 -2.5532204e-03 -1.2781023e-03  4.3092011e-03 -2.4154549e-03
 -7.5817830e-04 -3.9812718e-03 -4.3321415e-04  1.8063950e-03
 -1.3477367e-03 -7.1064574e-03 -2.0749094e-03 -1.6837581e-05
 -1.2061814e-03 -6.7349356e-06  2.7383112e-03 -1.1649758e-03
  1.8042199e-02 -8.7142229e-04 -5.8857142e-03  3.1167397e-03
 -2.3118199e-03 -4.0278430e-04 -5.8725630e-03  1.1314398e-02
 -8.2026469e-04 -1.4901583e-03 -1.0793941e-03 -6.7411823e-04
 -3.6991554e-05 -2.7007274e-03]


In [4]:
print(f"Best Explanation Method: anchor")
print(f"Explanation Score: 0.469047619047619")
print(f'''Feature Importance:
 ['LoanDuration' '>24.00']
 ['YearsAtCurrentJobGreaterOrEqualThan4' '0']
 ['CriticalAccountOrLoansElsewhere' '<=0']
 ['LoanRateAsPercentOfIncome' '>3.00']
 ['loanpurposeFurniture' '>0']
 ['loanpurposeUsedCar' '<=0']
 ['loanpurposeElectronics' '<=0']
 ['LoanAmount' '>3941.50']
 ['SavingsAccountBalanceGreaterOrEqualThan200' '<=0']
 ['HasGuarantor' '<=0']]''')

Best Explanation Method: anchor
Explanation Score: 0.469047619047619
Feature Importance:
 ['LoanDuration' '>24.00']
 ['YearsAtCurrentJobGreaterOrEqualThan4' '0']
 ['CriticalAccountOrLoansElsewhere' '<=0']
 ['LoanRateAsPercentOfIncome' '>3.00']
 ['loanpurposeFurniture' '>0']
 ['loanpurposeUsedCar' '<=0']
 ['loanpurposeElectronics' '<=0']
 ['LoanAmount' '>3941.50']
 ['SavingsAccountBalanceGreaterOrEqualThan200' '<=0']
 ['HasGuarantor' '<=0']]


In [5]:
print("\nAll Explanations:")
for idx, explanation in enumerate(mega_explanation):
    print(f"Explanation {idx + 1}:")
    print(f"  Type: {explanation.best_explanation_type}")
    print(f"  Score: {explanation.score}")
    print(f"  Features Importance: {explanation.list_exp}\n")


All Explanations:
Explanation 1:
  Type: anchor
  Score: 0.469047619047619
  Features Importance: [['LoanDuration' '>24.00']
 ['YearsAtCurrentJobGreaterOrEqualThan4' '0']
 ['CriticalAccountOrLoansElsewhere' '<=0']
 ['LoanRateAsPercentOfIncome' '>3.00']
 ['loanpurposeFurniture' '>0']
 ['loanpurposeUsedCar' '<=0']
 ['loanpurposeElectronics' '<=0']
 ['LoanAmount' '>3941.50']
 ['SavingsAccountBalanceGreaterOrEqualThan200' '<=0']
 ['HasGuarantor' '<=0']]
      
Explanation 2:
  Type: lime_0.25
  Score: 0.4667754962888392
  Features Importance: [ 4.3687555e-03 -1.9995717e-03  3.2969711e-03 -1.3142395e-03
  1.3283708e-02  5.1954640e-03  5.7685506e-03  3.6398688e-04
 -2.5532204e-03 -1.2781023e-03  4.3092011e-03 -2.4154549e-03
 -7.5817830e-04 -3.9812718e-03 -4.3321415e-04  1.8063950e-03
 -1.3477367e-03 -7.1064574e-03 -2.0749094e-03 -1.6837581e-05
 -1.2061814e-03 -6.7349356e-06  2.7383112e-03 -1.1649758e-03
  1.8042199e-02 -8.7142229e-04 -5.8857142e-03  3.1167397e-03
 -2.3118199e-03 -4.0278430e

In [1]:
print("\nAll Explanations:")
for idx, explanation in enumerate(mega_explanation):
    print(f"Explanation {idx + 1}:")
    print(f"  Type: {explanation.best_explanation_type}")
    print(f"  Score: {explanation.score}")
    print(f"  Features Importance: {explanation.list_exp}\n")


All Explanations:
Explanation 1:
  Type: anchor
  Score: 0.469047619047619
  Features Importance: [['Gender' '0']
 ['ForeignWorker' '0']
 ['Single' '0']
 ['Age' '0']
 ['LoanDuration' '0']
 ['LoanAmount' '0']
 ['LoanRateAsPercentOfIncome' '0']
 ['YearsAtCurrentHome' '0']
 ['NumberOfOtherLoansAtBank' '0']
 ['NumberOfLiableIndividuals' '0']
 ['HasTelephone' '0']
 ['CheckingAccountBalanceGreaterOrEqualTo0' '0']
 ['CheckingAccountBalanceGreaterOrEqualThan200' '0']
 ['SavingsAccountBalanceGreaterOrEqualThan200' '0']
 ['SavingsAccountBalanceGreaterOrEqualThan500' '0']
 ['MissedPayments' '0']
 ['NoCurrentLoan' '0']
 ['CriticalAccountOrLoansElsewhere' '0']
 ['OtherLoansAtBank' '0']
 ['OtherLoansAtStore' '0']
 ['HasCoapplicant' '0']
 ['HasGuarantor' '0']
 ['OwnsHouse' '0']
 ['RentsHouse' '0']
 ['Unemployed' '0']
 ['YearsAtCurrentJobLessThan1' '0']
 ['YearsAtCurrentJobGreaterOrEqualThan4' '0']
 ['JobClassIsSkilled' '0']
 ['loanpurposeBusiness' '0']
 ['loanpurposeEducation' '0']
 ['loanpurposeEle

### 3. Fairness Metric Calculation

* Configuration

In [ ]:
from mega_explainer.disparate_impact_calculator import DisparateImpactCalculator

* Getting Protected class

In [20]:
german_raw = pd.read_csv("c:\\Users\\Dell V3400\\OneDrive\\Tài liệu\\machine learning\\ML1 Project\\data\\german_raw.csv")

In [21]:
genders = german_raw['Gender']
genders

0        Male
1      Female
2        Male
3        Male
4        Male
        ...  
995    Female
996      Male
997      Male
998      Male
999      Male
Name: Gender, Length: 1000, dtype: object

In [ ]:
# Mapping function
gender_mapping = {'Male': 1, 'Female': 0}
genders = [gender_mapping[gender] for gender in genders]
print(genders)  # Output: [1, 0, 1, 0, 1]

[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 

* Calculate Disparate Impact Ratio from list of explainations

In [23]:
calculator = DisparateImpactCalculator(mega_explanation, genders, 15)

In [24]:
disparate_impact_ratios = calculator.calculate_disparate_impact()
print("Disparate Impact Ratios for each feature:", disparate_impact_ratios)

Non-numeric data encountered in explanation: [['Gender' '0']
 ['ForeignWorker' '0']
 ['Single' '0']
 ['Age' '0']
 ['LoanDuration' '0']
 ['LoanAmount' '0']
 ['LoanRateAsPercentOfIncome' '0']
 ['YearsAtCurrentHome' '0']
 ['NumberOfOtherLoansAtBank' '0']
 ['NumberOfLiableIndividuals' '0']
 ['HasTelephone' '0']
 ['CheckingAccountBalanceGreaterOrEqualTo0' '0']
 ['CheckingAccountBalanceGreaterOrEqualThan200' '0']
 ['SavingsAccountBalanceGreaterOrEqualThan200' '0']
 ['SavingsAccountBalanceGreaterOrEqualThan500' '0']
 ['MissedPayments' '0']
 ['NoCurrentLoan' '0']
 ['CriticalAccountOrLoansElsewhere' '0']
 ['OtherLoansAtBank' '0']
 ['OtherLoansAtStore' '0']
 ['HasCoapplicant' '0']
 ['HasGuarantor' '0']
 ['OwnsHouse' '0']
 ['RentsHouse' '0']
 ['Unemployed' '0']
 ['YearsAtCurrentJobLessThan1' '0']
 ['YearsAtCurrentJobGreaterOrEqualThan4' '0']
 ['JobClassIsSkilled' '0']
 ['loanpurposeBusiness' '0']
 ['loanpurposeEducation' '0']
 ['loanpurposeElectronics' '0']
 ['loanpurposeFurniture' '0']
 ['loanpu

In [25]:
biased_features = calculator.interpret_disparate_impact(threshold=0.8)
print("Potentially biased features (below 0.8 threshold):", biased_features)

Non-numeric data encountered in explanation: [['Gender' '0']
 ['ForeignWorker' '0']
 ['Single' '0']
 ['Age' '0']
 ['LoanDuration' '0']
 ['LoanAmount' '0']
 ['LoanRateAsPercentOfIncome' '0']
 ['YearsAtCurrentHome' '0']
 ['NumberOfOtherLoansAtBank' '0']
 ['NumberOfLiableIndividuals' '0']
 ['HasTelephone' '0']
 ['CheckingAccountBalanceGreaterOrEqualTo0' '0']
 ['CheckingAccountBalanceGreaterOrEqualThan200' '0']
 ['SavingsAccountBalanceGreaterOrEqualThan200' '0']
 ['SavingsAccountBalanceGreaterOrEqualThan500' '0']
 ['MissedPayments' '0']
 ['NoCurrentLoan' '0']
 ['CriticalAccountOrLoansElsewhere' '0']
 ['OtherLoansAtBank' '0']
 ['OtherLoansAtStore' '0']
 ['HasCoapplicant' '0']
 ['HasGuarantor' '0']
 ['OwnsHouse' '0']
 ['RentsHouse' '0']
 ['Unemployed' '0']
 ['YearsAtCurrentJobLessThan1' '0']
 ['YearsAtCurrentJobGreaterOrEqualThan4' '0']
 ['JobClassIsSkilled' '0']
 ['loanpurposeBusiness' '0']
 ['loanpurposeEducation' '0']
 ['loanpurposeElectronics' '0']
 ['loanpurposeFurniture' '0']
 ['loanpu

### 4. Selecting the most fair and faithful explanation

In [ ]:
from mega_explainer.fair_faithful_selector import FairFaithfulSelector

In [ ]:
selector = FairFaithfulSelector(
    sorted_explanations=mega_explanation,
    disparate_impact_ratios=disparate_impact_ratios,
    fairness_threshold=0.8        # Standard fairness ratio
)
result = selector.find_most_fair_and_faithful()

if result:
    print("Most Fair and Faithful Explanation:", result)
else:
    print("No explanation meets both the faithfulness and fairness criteria.")

Most Fair and Faithful Explanation: {'method_name': 'lime_0.25', 'faithfulness_score': 0.4667754962888392, 'explanation_data': array([ 4.3687555e-03, -1.9995717e-03,  3.2969711e-03, -1.3142395e-03,
        1.3283708e-02,  5.1954640e-03,  5.7685506e-03,  3.6398688e-04,
       -2.5532204e-03, -1.2781023e-03,  4.3092011e-03, -2.4154549e-03,
       -7.5817830e-04, -3.9812718e-03, -4.3321415e-04,  1.8063950e-03,
       -1.3477367e-03, -7.1064574e-03, -2.0749094e-03, -1.6837581e-05,
       -1.2061814e-03, -6.7349356e-06,  2.7383112e-03, -1.1649758e-03,
        1.8042199e-02, -8.7142229e-04, -5.8857142e-03,  3.1167397e-03,
       -2.3118199e-03, -4.0278430e-04, -5.8725630e-03,  1.1314398e-02,
       -8.2026469e-04, -1.4901583e-03, -1.0793941e-03, -6.7411823e-04,
       -3.6991554e-05, -2.7007274e-03], dtype=float32), 'is_fair': True}
